In [184]:
using CSV, DataFrames, Statistics, StatsBase, Distributions, JuMP, Gurobi, LinearAlgebra


#include("eval2.jl")
const GRB_ENV = Gurobi.Env()

Academic license - for non-commercial use only - expires 2022-09-11


Gurobi.Env(Ptr{Nothing} @0x00007fd5706c9800, false, 0)

In [185]:
import Pkg

include("src/algos/multi_armed_bandits.jl")
include("src/algos/passive_agressive.jl")
include("src/algos")
include("src/algos/OLS.jl")
include("src/algos/adaptive_linear_decision_rule.jl")
include("src/metrics.jl")

compute_CVaR (generic function with 1 method)

In [186]:
##### SAFI #####

X_test_adaptive = CSV.read("data/X_test_adaptive.csv", DataFrame)
y_test = CSV.read("data/y_test_speed.csv", DataFrame)

X = Matrix(X_test_adaptive)

#X = X[!,[2, 3,5,8,9,11,12,13,25]]
y = y_test[!, "speed"];

In [187]:
n, p = size(X)
#y_safi = (y .- mean(y[1:floor(Int, n/2),:], dims =1))./ std(y[1:floor(Int, n/2),:], dims=1);
split_index = floor(Int,n*0.5)

X = (X .- mean(y[1:floor(Int, split_index),:], dims =1))./ std(y[1:floor(Int, split_index),:], dims=1);#[!,1]

y = (y .- mean(y[1:floor(Int, split_index),:], dims =1))./ std(y[1:floor(Int, split_index),:], dims=1);

In [181]:
mean(y)

-0.01564420238613343

In [104]:
describe(abs.(X_test_adaptive[:,2:7] .- y), :mean, :std) 

LoadError: MethodError: no method matching describe(::Matrix{Float64}, ::Symbol, ::Symbol)
[0mClosest candidates are:
[0m  describe([91m::AbstractDataFrame[39m, ::Union{Symbol, Pair{var"#s15", var"#s14"} where {var"#s15"<:Union{Function, Type}, var"#s14"<:Union{AbstractString, Symbol}}}...; cols) at /Users/iai/builds/InterpretableAI/SystemImage/SysImgBuilder/.julia/packages/DataFrames/nxjiD/src/abstractdataframe/abstractdataframe.jl:570
[0m  describe(::Any) at /Users/iai/builds/InterpretableAI/SystemImage/SysImgBuilder/.julia/packages/StatsBase/DU1bT/src/scalarstats.jl:694

## Testing Linear Adaptive Rule

In [ ]:
Arguments:
  delta-inf  =>  0.01
  epsilon-l2  =>  0.01
  max-cuts  =>  10
  uncertainty  =>  0.0
  reg  =>  0.1
  benders  =>  false
  end-id  =>  8
  last_yT  =>  false
  filename-results  =>  results.csv
  num-past  =>  500
  fix-beta0  =>  false
  begin-id  =>  2
  verbose  =>  false
  delta-l2  =>  0.01
  epsilon-inf  =>  0.01
  rho  =>  1.0
  filename-X  =>  data/energy_predictions_test_val.csv
  more_data_for_beta0  =>  false
  val  =>  2000
  train_test_split  =>  0.5
  past  =>  10


4247

In [188]:
function get_X_Z_y(X, y, T)
    """
    Input: training data X and corresponding labels y ; how many time-steps from the past to be used
    Output: the past features X with past targets y as a Z training data (no present features)
    """
    n, p = size(X)
    #T past time steps * p features + T targets
    Z = ones(n-T, T*p+T)
    for i=T+1:n
        for t=1:T
            Z[i-T,1+p*(t-1):p*t] = X[i-t,:]
        end
        Z[i-T, (p*T+1):end] = y[i-T:i-1]
    end
    return X[T+1:end,:], Z, y[T+1:end]
end

get_X_Z_y (generic function with 1 method)

In [189]:
function adaptive_ridge_regression_exact_no_stable(X, y, ρ_β0, ρ_V0, T)

    #Version with actual robust equivalence
    #The formula for the regularization is different
    #no stable part
    # Create model
    model = Model(with_optimizer(Gurobi.Optimizer, GRB_ENV))
    set_optimizer_attribute(model, "OutputFlag", 0)
    X, Z, y = get_X_Z_y(X, y, T)

    N, P = size(X)
    # Add variables

    @variable(model, β0[j=1:P])
    @variable(model, V0[j=1:P, k=1:T*P+T])
    @variable(model, t>=0)
    @variable(model, β[t=1:N,j=1:P])

    # Add objective
    @objective(model, Min, t + 1/N*ρ_V0 * sum(β[t,k]^2 for t=1:N for k=1:P)
    )

    @constraint(model, t >= 1 / N * sum((y[i] - sum(transpose(X[i, :])*β[i,:]))^2 for i=1:N))

    #linear decision rule for adaptive RO
    @constraint(model, [i=1:N], β[i,:] .== β0.+V0*Z[i,:])

    optimize!(model);
    return objective_value(model), getvalue.(β0), getvalue.(V0), getvalue.(β)
end

adaptive_ridge_regression_exact_no_stable (generic function with 1 method)

In [190]:
split_ = 0.5
past = 5
num_past = 200

last_yT = false
max_cuts = 10
verbose = false

uncertainty = 0.#1
δ_inf = 0.02
ϵ_inf = 0.02

ϵ_l2 = 0.05
δ_l2 = 0.05

reg = 1/(past*num_past)
ρ = 0.1
val = 200; #n-split_index;

fix_β0 = false
more_data_for_β0 = false
benders = false
ridge = true

true

In [191]:
X0, y0, Xt, yt, yt_true, D_min, D_max = prepare_data_from_y(X, y, max(split_index-num_past*past+1, 1), num_past*past, val, uncertainty, last_yT)

([1.0 -0.5914705640897817 … -0.48004314729993747 -0.0983750566966326; 1.0 -0.4417684485207097 … -0.2052326411445407 -0.3515734892306719; … ; 1.0 0.8206847388166343 … 0.6200550034140118 0.8079779237889819; 1.0 0.7753563330298929 … 0.4375827493985346 -0.20553596264312277], [0.1785812337725732, 0.1216182572845958, 0.434914627968472, -0.8609930871330181, -0.22015960164327011, 0.4918776044564499, 0.7909332310183326, 1.0187851369702425, 0.7624517427743438, 0.5630813250664224  …  -0.4480115075951802, 0.07794664197714606, 0.5915628133104112, 1.0311271152093058, 0.5858665156616135, 0.34187509970477536, 0.043768856084359965, 1.077646879341154, 0.989354265784786, 0.8327060804428499], [1.0 0.4155672993742252 … 0.5388189441872216 0.25041525868630377; 1.0 0.07720078672347143 … 0.2808988695788197 0.25041525868630377; … ; 1.0 -1.7701708465302375 … -0.9756672867525845 -0.7956992387626771; 1.0 -1.4780046871489425 … -0.6466156373686673 -0.3873799419764161], [0.4804850091588551, -0.07965092630625874, -0.2

In [192]:
β_list_linear_adaptive_trained_one = zeros(val, p)
obj, β0_0, V0_0, β_list = adaptive_ridge_regression_exact_no_stable(X0, y0, ρ, ρ, past)
β_l2_init = l2_regression(X0,y0,ρ);

In [183]:
m = mean(y)
s = std(y)

-0.01564420238613343

In [154]:
X_ = vcat(X0,Xt)
y_true = vcat(y0,yt)*
err_baseline = [abs(y_true[s]-dot(X_[s,:],β_l2_init)) for s=past+1:size(y_true)[1]]
err_ARO = [abs(y_true[s]-dot(X_[s,:],β_list[s-past,:])) for s=past+1:size(y_true)[1]]

995-element Vector{Float64}:
 0.41368766221301867
 0.6002669249199575
 0.4154990596426551
 0.3789953803701659
 0.11719799355200483
 0.3559113788881998
 0.04548115962505043
 0.008093436071872573
 0.20378775955291173
 0.1450505745802862
 0.0854257891142867
 0.04657091989213058
 0.27128630993198244
 ⋮
 0.42133963212322667
 0.17159684578091217
 0.17584186304311344
 0.3113016263609841
 0.6048922261018194
 0.4657735447062539
 0.09104406690551559
 0.031198223436717465
 0.2007291194967188
 0.678925526007188
 0.1262300126684589
 0.08872995515489113

In [155]:
get_metrics(err_baseline, y_true[past+1:end])

println()
get_metrics(err_ARO, y_true[past+1:end])

MAE : 0.5041359819776762
MAPE : 149.78884366268463
RMSE : 0.6685874319782864
R2 : 0.583253934391076
CVAR 0.05 :1.8810033315411832
CVAR 0.15 :1.3129597634937162

MAE : 0.21264888457800998
MAPE : 74.19886024533643
RMSE : 0.27648240489830106
R2 : 0.9287327356790053
CVAR 0.05 :0.6992963361320879
CVAR 0.15 :0.5438955449925583


In [156]:
obj

0.11021965399644834

In [157]:
β_l2_init

8-element Vector{Float64}:
 -0.11441614737364723
  0.09204478185354534
  0.21394138557117062
  0.02247902739122068
  0.3374688857767796
 -0.17110401369707778
  0.3070029534582963
  0.1002186710745546

In [193]:
for s=1:val

        #TODO check split_index with max(split inex, 1)
        if more_data_for_β0
            X0, y0, Xt, yt, yt_true, D_min, D_max = prepare_data_from_y(X, y, max(split_index-num_past*past+1, 1), s+(num_past-1)*past, past-1, uncertainty, last_yT)
        else
            X0, y0, Xt, yt, yt_true, D_min, D_max = prepare_data_from_y(X, y, max(s+split_index-num_past*past+1, 1), (num_past-1)*past, past-1, uncertainty, last_yT)
        end

        #TODO: evaluate also if we change Vt regularly
        #obj, β_linear_adaptive_pure_0_Vt, Vt_adaptive_pure, _ = adaptive_ridge_regression_exact_Vt(vcat(X0,Xt), vcat(y0,yt), ρ, ρ, past, 1)


        #Line to get Z_{t+1}
        X_for_Z = X[split_index-past+s+1:split_index+s+1,:]
        X_for_Z[:,1] .= 1
        y_for_Z = y[split_index-past+s+1:split_index+s+1,:]
        X_, Z_test, y_ = get_X_Z_y(X_for_Z, y_for_Z, past)

        β_list_linear_adaptive_trained_one[s,:] = β0_0 + V0_0 * Z_test[1,:]
    println(Z_test)
    end

[1.0 0.7753563330298929 0.45865252434542486 0.755277884350446 0.34735490006760156 0.37745432083291053 0.4375827493985346 -0.20553596264312277 1.0 0.8206847388166343 0.6547273490486348 0.755277884350446 0.41348720688084883 0.5908843205759627 0.6200550034140118 0.8079779237889819 1.0 0.6872709308717272 0.560175928450653 0.755277884350446 0.32519390976876567 0.6347123385464758 0.701317968336191 0.25041525868630377 1.0 0.3570808089759219 0.677676820868017 0.6176411759838114 0.23710841359491056 0.8117002947496084 0.7320855528795616 0.628788618120537 1.0 0.34066747376074613 0.6037748210247859 0.25943493230731796 0.35334495578614783 0.7481790308827228 0.6580263308916109 0.628788618120537 0.34187509970477536 0.043768856084359965 1.077646879341154 0.989354265784786 0.8327060804428499]
[1.0 0.4155672993742252 0.36236572666873684 0.6176411759838114 0.40267870994416105 0.4508820961613795 0.5388189441872216 0.25041525868630377 1.0 0.7753563330298929 0.45865252434542486 0.755277884350446 0.347354900

[1.0 -0.7781447769061721 -0.2818737460741565 -0.3348002446047465 -0.18588110520805268 -0.1674336251726299 -0.3351230836515676 -0.3515734892306719 1.0 -0.48482538196116254 -0.2803982910571653 -0.29873204283204335 -0.22300148454025515 -0.1668791404696301 -0.30493489479651875 -0.3515734892306719 1.0 -0.2802252782524025 -0.21184870337367412 -0.13922505440857955 -0.1075366197915009 -0.08947936869404811 -0.18898381484168328 -0.3515734892306719 1.0 -0.3524678327733738 -0.23030641664507323 -0.13922505440857955 -0.1943218790640412 -0.15413589161803734 -0.2687950916602155 -0.3515734892306719 1.0 -0.4010890351738676 -0.061295100812137675 -0.13922505440857955 -0.06062225413386143 -0.08831432877899927 -0.19382243785696657 -0.3515734892306719 -0.53535473821008 -0.14231020044303377 -0.054966969828134545 -0.344528766975354 -0.5619374605711352]
[1.0 -0.9168275264964718 -0.21951085030404857 -0.061543334323586114 -0.2283148770611009 -0.17377993050217416 -0.3850618197321115 -0.3873799419764161 1.0 -0.7781

[1.0 1.518832774059358 0.6666438898039666 0.5302816757015703 0.7242106168797403 0.7274183086370757 0.7846089135264627 1.0430461785908465 1.0 1.6327213966037974 0.7580811444773504 0.5302816757015703 0.8138716520202759 0.9321946978539044 0.9027247864884625 1.0430461785908465 1.0 1.6363337887793035 1.0445939223574157 0.9254611781896724 0.8758892951898911 1.1262149044553553 1.0240308589446752 1.0430461785908465 1.0 1.3984280454662434 1.0736562620726526 0.9254611781896724 0.8270904803440761 1.1147788370658627 1.0842303007747194 0.8079779237889819 1.0 1.098649401740669 0.9799061966279036 0.6480186069248158 0.6858270835439167 0.9985267529590558 0.8732947754114222 0.8079779237889819 0.4747887115100575 0.7140332127595634 1.021633285794642 0.8298579316184506 0.9304925234138762]
[1.0 1.4500430957723407 0.7403808679043117 0.5302816757015703 0.5740793378258854 0.6369956505749679 0.7880289586027565 1.0430461785908465 1.0 1.518832774059358 0.6666438898039666 0.5302816757015703 0.7242106168797403 0.72

[1.0 0.8559872808637692 0.21485998512716578 0.43994077295270684 0.2065358144840483 0.5768185091010274 0.45112873351086 0.7157765636413992 1.0 1.1316448171558016 0.5314047834712227 0.6480186069248158 0.371438106008024 0.8608925515660751 0.7256559189488351 0.4212209472461236 1.0 1.053109160525019 0.34454628354888495 0.43994077295270684 0.2083647247701475 0.6747686476396029 0.5654714901487334 0.4212209472461236 1.0 0.8598156714455452 0.125927278604005 0.43994077295270684 0.06691283748078958 0.5173012261433539 0.4342272728783633 0.4212209472461236 1.0 0.6988694459307603 0.28379469949469227 0.43994077295270684 0.1364123967231708 0.5181237393927809 0.45107245009745445 0.25041525868630377 0.8127690386720561 0.19377136083603375 0.24124050790934876 -0.15465217868209627 -0.05876450159400006]
[1.0 0.40480802347045736 0.22919312319702345 0.11249048348169018 0.21411473850577367 0.44811634166379904 0.3094744474042703 0.23810319393027407 1.0 0.8559872808637692 0.21485998512716578 0.43994077295270684 

[1.0 1.2969957035592896 0.6028860846656204 0.9254611781896724 0.5162537709849505 0.6109077909642218 0.7452297927372803 0.8079779237889819 1.0 1.4070098389323715 0.8091215094109437 0.9254611781896724 0.5428566202643657 0.6608442684887479 0.8403812042265346 0.8079779237889819 1.0 1.469601784079509 0.8465572357809842 0.9254611781896724 0.7305632989066091 0.721703386736451 0.8546744462957601 1.0430461785908465 1.0 1.5460181497965075 0.9171626172859257 0.9254611781896724 0.664310230583207 0.8307467134620471 0.9360625522120165 1.0430461785908465 1.0 1.2659766291469272 0.8518801980818794 0.9254611781896724 0.5741893333334835 0.8624268516336917 0.8788144188609353 0.8079779237889819 0.8687826322185676 0.7937813798427308 0.47383932856859057 0.5459924321200285 0.33048250440718063]
[1.0 1.1299605353464433 0.6312861728162081 0.755277884350446 0.4284502415447108 0.5672977015393128 0.5799436540423095 0.8079779237889819 1.0 1.2969957035592896 0.6028860846656204 0.9254611781896724 0.5162537709849505 0.

...

[1.0 -0.5108599138998565 0.01049317386255426 -0.13922505440857955 -0.07348107644625755 -0.06458245179665247 -0.008286130676050588 -0.0983750566966326 1.0 -0.3268524518517032 0.24348445044213907 0.30289700197068337 0.18754361760426225 0.29365506767671035 0.24465024354899614 -0.0983750566966326 1.0 -0.13530065534340788 0.4276206897181163 0.30289700197068337 0.31451842138217745 0.4338607223674084 0.409420040784659 0.23810319393027407 1.0 0.09146961284689868 0.34930212245586667 0.30289700197068337 0.3401031713828229 0.37759007427706665 0.48132907033322847 0.23810319393027407 1.0 0.4389576565970764 0.452660873663537 0.6480186069248158 0.5262089055708719 0.417541300728825 0.5743009508340808 0.5625905293616243 0.3304825044071801 -0.10908179749171414 -0.4651004005415744 -0.6739646476641576 -0.7052942847325457]
[1.0 -0.39382245740681443 -0.12651930385421364 -0.13922505440857955 -0.28112194871046864 -0.21721848540339442 -0.1495688170669491 -0.0983750566966326 1.0 -0.5108599138998565 0.0104931738

[1.0 -1.1597151740825455 0.003415239219040615 -0.061543334323586114 -0.19377953456489908 -0.15529256992620621 -0.33639861355005296 -0.3873799419764161 1.0 -1.0243112650230326 0.16262875220711492 -0.061543334323586114 -0.06539518193378915 0.019531474761072386 -0.24006858883510468 0.09030929107340781 1.0 -1.120703794245383 -0.031629637101846055 -0.061543334323586114 -0.22769010713498075 -0.1410856331492158 -0.39069463963496026 -0.3873799419764161 1.0 -1.1610191163881758 -0.03190915442747247 -0.061543334323586114 -0.269878995152107 -0.2220522043980221 -0.4835693777181069 -0.3873799419764161 1.0 -0.7591566487046388 0.07228006001441642 -0.061543334323586114 -0.13465874356458063 -0.1253277747297609 -0.4040128366978487 -0.3515734892306719 0.30579854792905664 0.09218738609914091 0.07509849315274698 0.14820097964565163 0.34187509970477536]
[1.0 -1.3555889804041308 -0.012380138457263675 -0.061543334323586114 -0.2505503179702064 -0.2344658225848691 -0.39158631218062173 -0.3873799419764161 1.0 -1.

[1.0 -1.9201738608194294 -1.205626386146273 -0.993651652197808 -1.178107686130884 -1.4123024841041991 -1.174542031276302 -0.9358991367626334 1.0 -1.3756450817113783 -1.074550874653965 -0.993651652197808 -0.9397018507842106 -1.0733078558674916 -0.9043468007586216 -0.9358991367626334 1.0 -0.8282003371069949 -0.620398563340747 -0.5860414873751079 -0.7325320623355551 -0.6824339812862003 -0.5801273781316967 -0.3515734892306719 1.0 -0.44581628782444455 -0.43446912956492845 -0.13922505440857955 -0.5196877930580966 -0.37778985329537523 -0.34271431956255205 -0.3515734892306719 1.0 -0.3500956531964762 -0.4544236020065263 -0.13922505440857955 -0.478995493322239 -0.30409098184171673 -0.2846585253744609 -0.3515734892306719 -0.6407362447128383 -0.6103559905859169 -0.7081424335569445 -0.673015264722692 -0.9084622342063327]
[1.0 -1.9689003516334653 -1.258205150185877 -0.993651652197808 -0.9985089466730099 -1.33067596268695 -1.2284601456528232 -0.7956992387626771 1.0 -1.9201738608194294 -1.205626386146

[1.0 -0.28844420646618635 -0.07525478960813992 -0.29873204283204335 -0.028240340000811187 0.006749697994815873 -0.19504192415866714 -0.04895298934177866 1.0 -0.3471061119189492 -0.07177099092911159 -0.29873204283204335 -0.02051764142643021 0.0009792271426439585 -0.1620448915123558 -0.04895298934177866 1.0 -0.19189953221634792 0.08971973097002246 -0.29873204283204335 0.1768873827037268 0.1842750411791318 -0.002921495399628724 -0.04895298934177866 1.0 -0.0749057798763574 0.07746989983925934 -0.29873204283204335 0.2313944574234017 0.2503371079161878 0.08937982346908073 -0.04895298934177866 1.0 -0.08717295784701111 0.14482423338710324 -0.13922505440857955 0.11081579842341549 0.06775948304320341 0.0514534280652968 -0.04895298934177866 -0.13091760514543851 -0.25528677047752313 -0.5562411629223382 -0.5590893117467365 -0.5486460993906074]
[1.0 -0.40730380797074467 -0.3920467503071779 -0.29873204283204335 -0.3544684456069887 -0.26984443638929934 -0.3895627666565962 -0.3515734892306719 1.0 -0.28

[1.0 -0.2808712409461711 -0.7388292624233246 -0.524598542466654 -0.7450667653117347 -0.5842598038363608 -0.5219196172986312 -0.603140880734035 1.0 -0.1916167213112952 -0.5739118187476029 -0.524598542466654 -0.5578489998966876 -0.49932153093444726 -0.36173162711999024 -0.603140880734035 1.0 -0.13631873549306417 -0.3819291573123131 -0.29873204283204335 -0.09099195112247364 -0.30516184014922587 -0.2522106808842003 -0.603140880734035 1.0 -0.2558794827203815 -0.21751213338503836 -0.29873204283204335 -0.034651579967509595 -0.11444844763320812 -0.19730312997427374 -0.603140880734035 1.0 -0.3870455076162453 -0.29949267951677055 -0.29873204283204335 -0.09121518902732997 -0.05428529704013028 -0.16177472081172178 -0.603140880734035 -0.32174357638016293 -0.5514942482150065 -0.8496004918354224 -1.0081474430602932 -1.0869462272019959]
[1.0 -0.5586566970867177 -0.9276696182236314 -0.6261701875549548 -0.7511041003007657 -0.7462218292846804 -0.7915520858216636 -0.603140880734035 1.0 -0.2808712409461711

In [194]:
X0, y0, Xt, yt, yt_true, D_min, D_max = prepare_data_from_y(X, y, 1, split_index, val, uncertainty, last_yT)

([1.0 2.5883488445313474 … 1.5445165829128602 1.4886052778928418; 1.0 2.464610210027132 … 1.5258406114997327 1.4886052778928418; … ; 1.0 0.8206847388166343 … 0.6200550034140118 0.8079779237889819; 1.0 0.7753563330298929 … 0.4375827493985346 -0.20553596264312277], [1.3463222517761135, 1.4032852282640913, 1.33208150765412, 0.9618221604822652, 1.574174157728025, 1.5599334136060299, 1.1896740664341758, 0.7624517427743438, -0.049270672179337414, -0.3625670428632141  …  -0.4480115075951802, 0.07794664197714606, 0.5915628133104112, 1.0311271152093058, 0.5858665156616135, 0.34187509970477536, 0.043768856084359965, 1.077646879341154, 0.989354265784786, 0.8327060804428499], [1.0 0.4155672993742252 … 0.5388189441872216 0.25041525868630377; 1.0 0.07720078672347143 … 0.2808988695788197 0.25041525868630377; … ; 1.0 -1.7701708465302375 … -0.9756672867525845 -0.7956992387626771; 1.0 -1.4780046871489425 … -0.6466156373686673 -0.3873799419764161], [0.4804850091588551, -0.07965092630625874, -0.2419954092

In [175]:
Z_test

LoadError: UndefVarError: Z_test not defined

In [195]:
err_linear_adaptive_trained_one = [abs(yt_true[s]-dot(Xt[s,:],β_list_linear_adaptive_trained_one[s,:])) for s=1:val]
err_baseline_test = [abs(yt_true[s]-dot(Xt[s,:],β_l2_init)) for s=1:val]

200-element Vector{Float64}:
 0.2154477364884836
 0.14326466989862013
 0.03220382423807425
 0.08394761559681468
 0.1693244866519611
 0.1422071517814934
 0.2322810309846104
 0.2680961329039538
 0.21331168405623874
 0.24746898710156434
 0.05611452124807148
 0.1367099043534653
 0.11148401583877221
 ⋮
 0.03553654297749703
 0.03060052629759652
 0.24703635735238821
 0.5178816782982217
 0.4676768229915169
 0.40547307497029217
 0.1152200539557281
 0.15203379573462195
 0.6743032474271758
 0.6655172876053896
 0.678766534584951
 0.6519342475039283

In [196]:
get_metrics(err_baseline_test, yt_true)

println()
get_metrics(err_linear_adaptive_trained_one, yt_true)

MAE : 0.2947189530815509
MAPE : 175.6522130188673
RMSE : 0.37134802493097874
R2 : 0.5940451076455607

MAE : 0.2186778139021803
MAPE : 100.38096795197733
RMSE : 0.28723542150582415
R2 : 0.7571201232159726


## Synthetic

In [9]:
N0 = 900
Nt = 100
p = 10

δ = 0.5
δ_pert = 0.5
σ_pert = 1
#y = randn(N0+Nt)
n0 = floor(Int, size(y_safi)[1]/2)
y_true = y_safi[n0-N0:n0+Nt-1]
y = copy(y_true)
d = Uniform(-δ, δ)
biases = rand(d, p)
variances = rand(p)
gradual_after = false
gradual_before = true

#Create the features X by taking y and adding noise.
X = zeros(N0+Nt, p)

for i=1:p
    d = Normal(biases[i], variances[i])
    X[:,i] = y.+ rand(d, N0+Nt)
end

#### Perturbations on base learners because of data drift
d = Normal(0, δ_pert)
biases_perturb = rand(d, p)
d = Uniform(0, σ_pert)
variances_perturb = rand(d, p)

if gradual_after
    for i=1:p
        d = Normal(biases_perturb[i], variances_perturb[i])
        #We add the perturbation for each model 
        #We make the perturbations more and more intense across time
        X[N0+1:N0+Nt,i] = X[N0+1:N0+Nt,i].+rand(d, Nt).*[t/Nt for t=1:Nt]
    end
end

if gradual_before
    for i=1:p
        for t=1:Nt
            #We compute the perturbation for each model 
            #We make the perturbations more and more intense across time
            d = Normal(t/Nt*δ_pert, t/Nt*σ_pert)
            X[N0+t,i] = X[N0+t,i]+rand(d)
        end
    end
else
    for i=1:p
        d = Normal(biases_perturb[i], variances_perturb[i])
        #We add the perturbation for each model 
        #We make the perturbations more and more intense across time
        X[N0+1:N0+Nt,i] = X[N0+1:N0+Nt,i].+rand(d, Nt)
    end
end

In [10]:
#### Perturbations on y
perturb_y = true
perturb_y_norm = true
y_bias = -0.1
y_var = 0.5

if perturb_y
    if perturb_y_norm
        d = Normal(y_bias, y_var)
        y[N0+1:N0+Nt] = y[N0+1:N0+Nt] .+ rand(d, Nt)
    else
        d = Uniform(-y_bias, y_bias)
        y[N0+1:N0+Nt] = y[N0+1:N0+Nt] .+ rand(d, Nt)
    end
end
;

In [29]:
split_ = N0/(N0+Nt)
past = 20
num_past = floor(Int, Nt/past)

last_yT = false
max_cuts = 10
verbose = false

uncertainty = 0.#1
δ_inf = 0.02
ϵ_inf = 0.02

ϵ_l2 = 0.05
δ_l2 = 0.05

reg = 1/(past*num_past)
ρ = 0.1
val = Nt-1; #n-split_index;

fix_β0 = false
more_data_for_β0 = false
benders = false
ridge = true

eval_method(X, y, y_true, split_, past, num_past, val, uncertainty, ϵ_inf, δ_inf, last_yT, 
        ϵ_l2, δ_l2, ρ, reg, max_cuts, verbose, 
        fix_β0, more_data_for_β0, benders, ridge)


### Mean Baseline ###
MAE : 0.42345111861420726
MAPE : 187.57814003246975
RMSE : 0.5017148222829678
R2 : 0.2684574206577698
CVAR 0.05 :1.1315406489739686
CVAR 0.15 :0.8671868300356882

### Bandits Full Baseline ###
MAE : 0.35048051123948665
MAPE : 163.8983188941456
RMSE : 0.5198624052014562
R2 : 0.21457889059766233
CVAR 0.05 :1.4609555545070267
CVAR 0.15 :1.1120710262538487

### Bandits Only Last T Baseline ###
MAE : 0.3664926282734096
MAPE : 176.46620187337854
RMSE : 0.5475155488397699
R2 : 0.1287984082660898
CVAR 0.05 :1.4955622544668161
CVAR 0.15 :1.1764829878349519

### Passive-Aggressive Baseline ###
MAE : 0.5103165968166972
MAPE : 193.79379347546438
RMSE : 0.6738334814289204
R2 : -0.3195653975318984
CVAR 0.05 :1.644262319552229
CVAR 0.15 :1.2777854831230044

### β0 Baseline ###
MAE : 0.21725233606432692
MAPE : 74.74218826820477
RMSE : 0.3017689156644273
R2 : 0.7353482619067152
CVAR 0.05 :0.8529027877143107
CVAR 0.15 :0.6180074087399314

### βt Linear Decision Rule Adaptive with 

# Linear rule

In [24]:
function adaptive_ridge_regression_standard(X, y, ρ_β0, ρ_V0, T)
    
#     Adaptive ridge: does not run fast
    

    # Create model
    model = Model(with_optimizer(Gurobi.Optimizer, GRB_ENV))
    set_optimizer_attribute(model, "OutputFlag", 0)
    X, Z, y = get_X_Z_y(X, y, T)

    N, P = size(X)
    # Add variables

    @variable(model, β0[j=1:P])
    @variable(model, V0[j=1:P, k=1:T*P+T])
    @variable(model, t>=0)

    # Add objective
    @objective(model, Min, t + ρ_β0 * sum(β0[j]^2 for j=1:P)
                            + ρ_V0 * sum(V0[j,k]^2 for j=1:P for k=1:T*P+T)
    )

    @constraint(model, t >= 1 / N * sum((y[i] - sum(X[i, j] * (β0[j]
                 + sum(V0[j, l] * Z[i, l] for l=1:(T * P + T))
                 )
                for j=1:P))^2
        for i=1:N))

    optimize!(model);
    return objective_value(model), getvalue.(β0), getvalue.(V0), Z
end

adaptive_ridge_regression_standard (generic function with 1 method)

In [174]:
function adaptive_ridge_regression_before_exact(X, y, ρ_β0, ρ_V0, T)
    
    #Version with actual robust equivalence
    #We use beta t
    
    # Create model
    model = Model(with_optimizer(Gurobi.Optimizer, GRB_ENV))
    set_optimizer_attribute(model, "OutputFlag", 1)
    X, Z, y = get_X_Z_y(X, y, T)

    N, P = size(X)
    # Add variables

    @variable(model, β0[j=1:P])
    @variable(model, V0[j=1:P, k=1:T*P+T])
    @variable(model, t>=0)
    @variable(model, β[t=1:N,j=1:P])

    # Add objective
    @objective(model, Min, t + ρ_β0 * sum(β0[j]^2 for j=1:P)
                            + ρ_V0 * sum(V0[j,k]^2 for j=1:P for k=1:T*P+T)
    )

    @constraint(model, t >= 1 / N * sum((y[i] - sum(transpose(X[i, :])*β[i,:]))^2 for i=1:N))
        
    @constraint(model, [i=1:N], β[i,:] .== β0+V0*Z[i,:])

    optimize!(model);
    return objective_value(model), getvalue.(β0), getvalue.(V0), getvalue.(β)
end

adaptive_ridge_regression_before_exact (generic function with 1 method)

In [188]:
function adaptive_ridge_regression_exact(X, y, ρ_β0, ρ_V0, T, N0)
    
    #Version with actual robust equivalence
    #The formula for the regularization is different
    
    # Create model
    model = Model(with_optimizer(Gurobi.Optimizer, GRB_ENV))
    set_optimizer_attribute(model, "OutputFlag", 0)
    X, Z, y = get_X_Z_y(X, y, T)

    N, P = size(X)
    # Add variables

    @variable(model, β0[j=1:P])
    @variable(model, V0[j=1:P, k=1:T*P+T])
    @variable(model, t>=0)
    @variable(model, β[t=1:N,j=1:P])

    # If no stable part, then no reg
    if N0 == 1
        ρ_β0 = 0
    end
    
    # Add objective
    @objective(model, Min, t + 1/N0*ρ_β0 * sum(β0[j]^2 for j=1:P)
                            + 1/(N-N0)*ρ_V0 * sum(β[t,k]^2 for t=1:N for k=1:P)
    )

    @constraint(model, t >= 1 / N * sum((y[i] - sum(transpose(X[i, :])*β[i,:]))^2 for i=1:N))
    
    #stable and adaptive part
    @constraint(model, [i=N0:N], β[i,:] .== β0+V0*Z[i,:])
    @constraint(model, [i=1:N0-1], β[i,:] .== β0)

    optimize!(model);
    return objective_value(model), getvalue.(β0), getvalue.(V0), getvalue.(β)
end

adaptive_ridge_regression_exact (generic function with 2 methods)

In [12]:
function get_X_Z_y(X, y, T)
    """
    Input: training data X and corresponding labels y ; T: how many time-steps from the past to be used
    Output: the past features X with past targets y as a Z training data (no present features)
    """
    n, p = size(X)
    #T past time steps * p features + T targets
    Z = ones(n-T, T*p+T)
    for i=T+1:n
        for t=1:T
            Z[i-T,1+p*(t-1):p*t] = X[i-t,:]
        end
        Z[i-T, (p*T+1):end] = y[i-T:i-1]
    end
    return X[T+1:end,:], Z, y[T+1:end]
end

get_X_Z_y (generic function with 1 method)

In [13]:
e, Z, f = get_X_Z_y(X[1:5,1:2], y[1:5], 3)

([0.953041262548516 0.4782870249686575; 1.0535457742742587 0.334867774903859], [0.6507019593290054 0.4323663266396612 … -0.24864108988725883 0.4349146279684725; 0.953041262548516 0.4782870249686575 … 0.4349146279684725 0.7909332310183326], [0.7909332310183326, 0.7909332310183326])

In [164]:
obj, β0, V0, Z = adaptive_ridge_regression_standard(X[1:100,1:3], y[1:100], 0.1, 0.1, 3)
println(obj)

0.019454798289416986


In [165]:
β0+V0*Z[1,:]

3-element Vector{Float64}:
 0.1504249753800681
 0.4548341433068424
 0.14489314118672222

In [191]:
obj, β0, V0, β = adaptive_ridge_regression_exact(X[1:100,1:3], y[1:100], 0, 0.1, 3, 16)
println(obj)

0.5860827402650841


In [187]:
obj, β0, V0, β = adaptive_ridge_regression_exact(X[1:100,1:3], y[1:100], 0, 0.1, 3, 0)
println(obj)

0.5852809828471135


In [177]:
obj, β0, V0, β = adaptive_ridge_regression_before_exact(X[1:100,1:3], y[1:100], 0.1, 0.1, 3)
println(obj)
println(β0+V0*Z[1,:])
println(β[1,:])

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 291 rows, 331 columns and 4074 nonzeros
Model fingerprint: 0x6c09b65a
Model has 39 quadratic objective terms
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [1e-02, 3e+00]
  QMatrix range    [1e-06, 8e-02]
  QLMatrix range   [6e-05, 1e+00]
  Objective range  [1e+00, 1e+00]
  QObjective range [2e-01, 2e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
  QRHS range       [8e-01, 8e-01]
Presolve time: 0.00s
Presolved: 391 rows, 432 columns, 4759 nonzeros
Presolved model has 2 second-order cone constraints
Ordering time: 0.00s

Barrier statistics:
 Dense cols : 41
 Free vars  : 291
 AA' NZ     : 4.854e+03
 Factor NZ  : 1.827e+04
 Factor Ops : 8.039e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual    

In [170]:
_, β0, V0, β, X1, Z1 = adaptive_ridge_regression_exact_test(X[1:100,1:3], y[1:100], 0.1, 0.1, 3)
β[1,:]

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 291 rows, 331 columns and 4074 nonzeros
Model fingerprint: 0xac638d92
Model has 39 quadratic objective terms
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [1e-02, 3e+00]
  QMatrix range    [2e-01, 1e+01]
  QLMatrix range   [6e-01, 4e+00]
  Objective range  [1e+00, 1e+00]
  QObjective range [2e-01, 2e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
  QRHS range       [8e-01, 8e-01]
Presolve removed 291 rows and 291 columns
Presolve time: 0.00s
Presolved: 42 rows, 83 columns, 864 nonzeros
Presolved model has 2 second-order cone constraints
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 8.600e+02
 Factor NZ  : 9.030e+02
 Factor Ops : 2.558e+04 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual

3-element Vector{Float64}:
 0.1504249753800681
 0.45483414330684235
 0.14489314118672222

In [157]:
N, P = size(X1)
i = 1
T = 3
sum(X1[i, j] * (β0[j]+ sum(V0[j, l] * Z1[i, l] for l=1:(T * P + T))) for j=1:P)

0.533767797757831

In [151]:
transpose(X1[i,:])*β[i,:]

0.533767797757831

In [156]:
j = 1
println(β0[j]+sum(V0[j, l] * Z[i, l] for l=1:(T * P + T)))
println(β0+V0*Z[i,:])
println()

0.1504249753800681
[0.1504249753800681, 0.4548341433068424, 0.14489314118672222]


In [16]:
function evaluate_adaptive(X, y, y_true, β0, V0, T)
    X, Z, y = get_X_Z_y(X, y, T)
    
    N, P = size(X)

    pred = [sum(X[i, j] * (β0[j] + sum(V0[j, :] .* Z[i, :])) for j=1:P) for i=1:N]
    if size(y_true) == size(pred)
        err = [abs(y_true[i]-pred[i]) for i=1:N]
    else
        y_true = y_true[T+1:end]
        err = [abs(y_true[i]-pred[i]) for i=1:N]
    end
    return err
end

evaluate_adaptive (generic function with 1 method)

In [195]:
function evaluate_adaptive_retrained(X, y, y_true, β0_list, V0_list, T)
    X, Z, y = get_X_Z_y(X, y, T)
    
    N, P = size(X)

    pred = [sum(X[i, j] * (β0_list[i,j] + sum(V0_list[i, j, :] .* Z[i, :])) for j=1:P) for i=1:N]
    if size(y_true) == size(pred)
        err = [abs(y_true[i]-pred[i]) for i=1:N]
    else
        y_true = y_true[T+1:end]
        err = [abs(y_true[i]-pred[i]) for i=1:N]
    end
    return err
end

evaluate_adaptive_retrained (generic function with 1 method)

In [435]:
pred, err = evaluate_adaptive(X[1:100,1:3], y[1:100], y[4:100], β0, V0, 3)

([0.2772791481270517, -0.22139104249186187, 0.25353125721465, 0.13052228762534424, 0.16184232903639695, 0.02133275894336177, -0.23829147212091226, 0.14926772611852196, 0.28826028113700647, 0.7460128800839689  …  0.6974694989745849, 0.4334507651952026, 0.4535167025297599, 0.4228111533412082, -0.9470370564072363, -0.004111494312688896, 0.12274140243903106, 0.08108513878897414, -1.4025705199739362, 0.08251350395763898], [0.7111257347162706, 0.768332857553357, 0.1994216466416816, 0.6812973681052467, 0.08794262534614797, 0.24054297764516547, 0.02174221306394139, 0.4729100683816176, 0.04127284032870793, 0.5794229469798875  …  0.29188476681020137, 0.3992553152476475, 0.02696830662909544, 0.5024620796474668, 0.7050416471102421, 0.23408638321844036, 0.4254973199898675, 0.7075806393798176, 0.9146849288371715, 0.6178682421677187])

In [196]:
function eval_method(X, y, y_true, split_, past, num_past, val, uncertainty, ϵ_inf, δ_inf, last_yT,
        ϵ_l2, δ_l2, ρ, reg, max_cuts, verbose,
        fix_β0, more_data_for_β0, benders, ridge, linear_adapt)

    threshold_benders = 0.01
    n, p = size(X)
    split_index = floor(Int,n*split_)
    #TODO change spli_index with max(split inex, 1)
    X0, y0, Xt, yt, yt_true, D_min, D_max = prepare_data_from_y(X, y, split_index-num_past*past+1, num_past*past, val, uncertainty, last_yT)

    β_list0 = zeros(val, p)
    β_listt = zeros(val, p)
    β_listl2 = zeros(val, p)
    β_l2_init = l2_regression(X0,y0,ρ);
    β0_list_linear_adapt = zeros(val, p)
    V0_list_linear_adapt = zeros(val, p, (past-1)*p+past-1)
    err = ones(val)
    
    ###Linear decision rule
    _, β0_0, V0_0 = adaptive_ridge_regression_standard(X0, y0, ρ, ρ, past-1)
    print(size(V0))
    
    for s=1:val
        #TODO Check how much I use from the past
        if more_data_for_β0
            X0, y0, Xt, yt, yt_true, D_min, D_max = prepare_data_from_y(X, y, split_index-num_past*past+1, s+(num_past-1)*past, past-1, uncertainty, last_yT)
        else
            X0, y0, Xt, yt, yt_true, D_min, D_max = prepare_data_from_y(X, y, s+split_index-num_past*past+1, (num_past-1)*past, past-1, uncertainty, last_yT)
        end

        if benders
            ##TODO handle fix_beta0
            obj, β0_val, α, y_val = master_problem(X0, Xt, y0, D_min, D_max, threshold_benders, ϵ_inf, δ_inf, reg, ρ, max_cuts, verbose)
            _, βt_val = S_primal(Xt, y_val, β0_val, ϵ_inf, δ_inf);
        else
            if ridge
                obj, βt_val, β0_val = master_primal_l2_ridge(X0, Xt, y0, D_min, D_max, ϵ_inf, δ_inf, reg, ρ, ϵ_l2, δ_l2, fix_β0, β_l2_init)
            else
                obj, βt_val, β0_val = master_primal_l2(X0, Xt, y0, D_min, D_max, ϵ_inf, δ_inf, reg, ρ, ϵ_l2, δ_l2, fix_β0, β_l2_init)
            end
        end
        
        if linear_adapt
            #err[s] = evaluate_adaptive(vcat(X0[end-past+2:end,:],Xt), vcat(y0[end-past+2:end,:],yt), yt_true, β0, V0, past)
            _, β0, V0 = adaptive_ridge_regression_standard(vcat(X0,Xt), vcat(y0,yt), ρ, ρ, past-1)
            β0_list_linear_adapt[s,:] = β0
            V0_list_linear_adapt[s,:,:] = V0
            N = size(X0)[1]
        end
        
        β_listt[s,:] = βt_val[past-1,:]
        β_list0[s,:] = β0_val
        β_l2 = l2_regression(vcat(X0,Xt),vcat(y0,yt),ρ);
        β_listl2[s,:] = β_l2

    end

    X0, y0, Xt, yt, _, D_min, D_max = prepare_data_from_y(X, y, 1, split_index, val, uncertainty, last_yT)
    _, _, _, _, yt_true, _, _ = prepare_data_from_y(X, y_true, 1, split_index, val, uncertainty, last_yT)

    N = size(X0)[1]
    err_linear_rule = evaluate_adaptive(vcat(X0[N-past+2:end,:],Xt), vcat(y0[N-past+2:end,:],yt), yt_true, β0_0, V0_0, past-1)
    err_linear_rule_retrained = evaluate_adaptive_retrained(vcat(X0[N-past+2:end,:],Xt), vcat(y0[N-past+2:end,:],yt), 
        yt_true, β0_list_linear_adapt, V0_list_linear_adapt, past-1)
    
    err_0 = [abs(yt_true[s]-dot(Xt[s,:],β_list0[s,:])) for s=1:val]
    err_t = [abs(yt_true[s]-dot(Xt[s,:],β_listt[s,:])) for s=1:val]
    err_baseline = [abs(yt_true[s]-dot(Xt[s,:],β_l2_init)) for s=1:val]
    err_l2 = [abs(yt_true[s]-dot(Xt[s,:],β_listl2[s,:])) for s=1:val]

    println("\n### β0 Baseline ###")
    println("MAE Baseline: ", mean(err_baseline))
    println("CVAR 0.05 :", compute_CVaR(err_baseline, 0.05))
    println("CVAR 0.15 :", compute_CVaR(err_baseline, 0.15))
    println("R2 : ", R2_err(err_baseline, yt_true))

    println("\n### β0 Baseline Retrained ###")
    println("MAE Baseline: ", mean(err_l2))
    println("CVAR 0.05 :", compute_CVaR(err_l2, 0.05))
    println("CVAR 0.15 :", compute_CVaR(err_l2, 0.15))
    println("R2 : ", R2_err(err_l2, yt_true))
    
    println("\n### β0 V0 Linear rule Adaptive ###")
    println("MAE Baseline: ", mean(err_linear_rule))
    println("CVAR 0.05 :", compute_CVaR(err_linear_rule, 0.05))
    println("CVAR 0.15 :", compute_CVaR(err_linear_rule, 0.15))
    println("R2 : ", R2_err(err_linear_rule, yt_true))
    
    println("\n### β0 V0 Linear rule Adaptive Retrained ###")
    println("MAE Baseline: ", mean(err_linear_rule_retrained))
    println("CVAR 0.05 :", compute_CVaR(err_linear_rule_retrained, 0.05))
    println("CVAR 0.15 :", compute_CVaR(err_linear_rule_retrained, 0.15))
    println("R2 : ", R2_err(err_linear_rule_retrained, yt_true))

    println("\n### β0 Adaptive ###")
    println("MAE 0: ", mean(err_0))
    println("CVAR 0.05 :", compute_CVaR(err_0, 0.05))
    println("CVAR 0.15 :", compute_CVaR(err_0, 0.15))
    println("R2 : ", R2_err(err_0, yt_true))

    println("\n### βt Adaptive ###")
    println("MAE t: ", mean(err_t))
    println("CVAR 0.05 :", compute_CVaR(err_t, 0.05))
    println("CVAR 0.15 :", compute_CVaR(err_t, 0.15))
    println("R2 : ", R2_err(err_t, yt_true))
end

eval_method (generic function with 2 methods)

In [198]:
linear_adapt = true
split_ = N0/(N0+Nt)
past = 5
num_past = 20#floor(Int, Nt/past)

last_yT = false
max_cuts = 10
verbose = false

uncertainty = 0.#1
δ_inf = 0.02
ϵ_inf = 0.02

ϵ_l2 = 0.05
δ_l2 = 0.05

reg = 1/(past*num_past)
ρ = 0.1
val = Nt-1; #n-split_index;

fix_β0 = false
more_data_for_β0 = false
benders = false
ridge = true

eval_method(X, y, y_true, split_, past, num_past, val, uncertainty, ϵ_inf, δ_inf, last_yT, 
        ϵ_l2, δ_l2, ρ, reg, max_cuts, verbose, 
        fix_β0, more_data_for_β0, benders, ridge, linear_adapt)

(3, 12)

LoadError: Gurobi Error 10020: Constraint Q not PSD (diagonal adjustment of 1.1e-05 would be required). Set NonConvex parameter to 2 to solve model.